# FIT3164 Data Science Project

In [1]:
import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import sklearn
from statsmodels.tsa.seasonal import seasonal_decompose

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

## Import data

In [2]:
"""
from pathlib import Path  
df_true= pd.read_csv("./data/cleaned_actual.csv")
df_pred= pd.read_csv("./data/cleaned_forecasts.csv")
df_true = df_act.rename(columns={"Time": 'time', 
                                 "Load (kW)": "load_kw_true", 
                                 "Pressure_kpa": "pres_kpa_true",
                                 'Cloud Cover (%)': 'cld_pct_true',
                                 'Humidity (%)': 'hmd_pct_true',
                                 'Temperature (C)': 'temp_c_true',
                                 'Wind Direction (deg)': 'wd_deg_true',
                                 'Wind Speed (kmh)':'ws_kmh_true'})
df_pred = df_pred.rename(columns={"Time": 'time', 
                                 "Load (kW)": "load_kw_pred", 
                                 "Pressure (kpa)": "pres_kpa_pred",
                                 'Cloud Cover (%)': 'cld_pct_pred',
                                 'Humidity (%)': 'hmd_pct_pred',
                                 'Temperature (C)': 'temp_c_pred',
                                 'Wind Direction (deg)': 'wd_deg_pred',
                                 'Wind Speed (kmh)':'ws_kmh_pred'})
result = pd.merge(df_true, df_pred, on="time")
result['time']= pd.to_datetime(result['time'])
result = result.set_index('time')
result.head()


filepath = Path('./data/data_cleaned.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
result.to_csv(filepath)  
"""

'\nfrom pathlib import Path  \ndf_true= pd.read_csv("./data/cleaned_actual.csv")\ndf_pred= pd.read_csv("./data/cleaned_forecasts.csv")\ndf_true = df_act.rename(columns={"Time": \'time\', \n                                 "Load (kW)": "load_kw_true", \n                                 "Pressure_kpa": "pres_kpa_true",\n                                 \'Cloud Cover (%)\': \'cld_pct_true\',\n                                 \'Humidity (%)\': \'hmd_pct_true\',\n                                 \'Temperature (C)\': \'temp_c_true\',\n                                 \'Wind Direction (deg)\': \'wd_deg_true\',\n                                 \'Wind Speed (kmh)\':\'ws_kmh_true\'})\ndf_pred = df_pred.rename(columns={"Time": \'time\', \n                                 "Load (kW)": "load_kw_pred", \n                                 "Pressure (kpa)": "pres_kpa_pred",\n                                 \'Cloud Cover (%)\': \'cld_pct_pred\',\n                                 \'Humidity (%)\': \'hm

## Inspect data

In [3]:
df = pd.read_csv("./data/data_cleaned.csv")
df = df.set_index('time')
df

FileNotFoundError: [Errno 2] No such file or directory: './data/data_cleaned.csv'

In [ ]:
df['load_kw_true'].mean()

In [ ]:
df.describe().transpose()

In [ ]:
sns.violinplot(df, orient="h", palette="Blues")

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(df["load_kw_true"])

In [ ]:
sns.heatmap(df.corr(), cmap="Blues", annot=True) #correlationship

### Min-Max Scaling (Normalization)

In [ ]:
df_norm=(df-df.min())/(df.max()-df.min()) #min max normalization

In [ ]:
sns.heatmap(df_norm.corr(), cmap="Blues", annot=True)

In [ ]:
sns.violinplot(df_norm, orient="h", palette="Blues")

### Z-Score (Standardization)

In [ ]:
df_std=(df-df.mean())/df.std() #mean normalization

In [ ]:
df_std

In [ ]:
sns.heatmap(df_std.corr(), cmap="Blues", annot=True)

In [ ]:
sns.violinplot(df_std, orient="h", palette="Blues")

In [ ]:
#***need to remove outliers?

### Split Data Frame

In [ ]:
df_denorm=df
df = df_norm[['load_kw_true']]
#df = df_std[['load_kw_true', 'Temperature (C) ', 'Wind Speed (kmh)']]

In [ ]:
#data after 20th June 2020
"""
column_indices = {name: i for i, name in enumerate(df.columns)}
df = df[int(len(df)*0.85):]
n = len(df)

train_df = df[0:int(n*0.7)]
val_df = df[int(n*0.7):int(n*0.9)]
test_df = df[int(n*0.9):]

num_features = df.shape[1]
"""

In [ ]:
df

In [ ]:
#take in all data

column_indices = {name: i for i, name in enumerate(df.columns)}

n = len(df)
n_test = 24*7

train_df = df[0:int(n*0.8)]
val_df = df[int(n*0.8):int(n-n_test)]
test_df = df[int(n*-n_test):]

num_features = df.shape[1]

### Data Windowing

In [ ]:
class WindowGenerator():
    def __init__(self, input_width, label_width, shift, 
                 train_df=train_df, val_df=val_df, 
                 test_df=test_df, label_columns=None):
        # Store the raw data.
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df

        # Work out the label column indices.
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in enumerate(label_columns)}
        self.column_indices = {name: i for i, name in enumerate(train_df.columns)}

        # Work out the window parameters.
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def __repr__(self):
        return '\n'.join([
            f'Total window size: {self.total_window_size}',
            f'Input indices: {self.input_indices}',
            f'Label indices: {self.label_indices}',
            f'Label column name(s): {self.label_columns}'])

    # 2. split window
    def split_window(self, features):
        inputs = features[:, self.input_slice, :]
        labels = features[:, self.labels_slice, :]
        if self.label_columns is not None:
            labels = tf.stack(
                [labels[:, :, self.column_indices[name]] for name in self.label_columns],
                axis=-1)

        # Slicing doesn't preserve static shape information, so set the shapes
        # manually. This way the `tf.data.Datasets` are easier to inspect.
        inputs.set_shape([None, self.input_width, None])
        labels.set_shape([None, self.label_width, None])

        return inputs, labels


    # 3. plot
    def plot(self, model=None, plot_col='load_kw_true', max_subplots=3):
        inputs, labels = self.example
        plt.figure(figsize=(12, 8))
        plot_col_index = self.column_indices[plot_col]
        max_n = min(max_subplots, len(inputs))
        for n in range(max_n):
            plt.subplot(max_n, 1, n+1)
            plt.ylabel(f'{plot_col} [normed]')
            plt.plot(self.input_indices, inputs[n, :, plot_col_index],
                     label='Inputs', marker='.', zorder=-10)

            if self.label_columns:
                label_col_index = self.label_columns_indices.get(plot_col, None)
            else:
                label_col_index = plot_col_index

            if label_col_index is None:
                continue

            plt.scatter(self.label_indices, labels[n, :, label_col_index],
                        edgecolors='k', label='Labels', c='#2ca02c', s=64)
            if model is not None:
                predictions = model(inputs)
                plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                            marker='X', edgecolors='k', label='Predictions',
                            c='#ff7f0e', s=64)

            if n == 0:
                plt.legend()

        plt.xlabel('Time [h]')
        
    @property
    def example(self):
        """Get and cache an example batch of `inputs, labels` for plotting."""
        result = getattr(self, '_example', None)
        if result is None:
            # No example batch was found, so get one from the `.train` dataset
            result = next(iter(self.train))
            # And cache it for next time
            self._example = result
        return result


    # 4. make dataset
    def make_dataset(self, data):
        data = np.array(data, dtype=np.float32)
        ds = tf.keras.utils.timeseries_dataset_from_array(
            data=data,
            targets=None,
            sequence_length=self.total_window_size,
            sequence_stride=1,
            shuffle=True,
            batch_size=32,)
        ds = ds.map(self.split_window)

        return ds


    @property
    def train(self):
        return self.make_dataset(self.train_df)

    @property
    def val(self):
        return self.make_dataset(self.val_df)

    @property
    def test(self):
        return self.make_dataset(self.test_df)




### Compile and Fit

In [ ]:
MAX_EPOCHS = 10

def compile_and_fit(model, window, patience=2):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                      patience=patience,
                                                      mode='min')

    model.compile(loss=tf.keras.losses.MeanSquaredError(),
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=[tf.keras.metrics.MeanAbsoluteError()])

    history = model.fit(window.train, epochs=MAX_EPOCHS,
                        validation_data=window.val,
                        callbacks=[early_stopping])
    return history


In [ ]:
def plot_history(H):
    x = [int(i+1) for i in range(len(H[list(H)[0]]))] 
    plt.figure(figsize=(12,4))
    plt.plot(x, H["val_loss"], label="val_loss")
    plt.plot(x, H["loss"], label="loss")
    plt.plot(x, H["val_mean_absolute_error"], label="val_mean_absolute_error")
    plt.plot(x, H["mean_absolute_error"], label="mean_absolute_error")
    plt.legend()
    plt.xticks(np.arange(min(x), max(x)+1, 1.0))

### BASELINE

In [ ]:
class RepeatBaseline(tf.keras.Model):
    def call(self, inputs):
        return inputs

OUT_STEPS = 48
multi_window = WindowGenerator(input_width=48,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS)

repeat_baseline = RepeatBaseline()
repeat_baseline.compile(loss=tf.keras.losses.MeanSquaredError(),
                        metrics=[tf.keras.metrics.MeanAbsoluteError()])

multi_val_performance ={}
multi_performance={}
multi_val_performance['Repeat'] = repeat_baseline.evaluate(multi_window.val)
multi_performance['Repeat'] = repeat_baseline.evaluate(multi_window.test, verbose=0)
multi_window.plot(repeat_baseline)


### LSTM

In [ ]:
#grid search
units = [64, 32]


In [ ]:
OUT_STEPS = 24*2
multi_window = WindowGenerator(input_width=24*7,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS, label_columns=['load_kw_true'])
multi_lstm_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units].
    # Adding more `lstm_units` just overfits more quickly.
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.LSTM(64, return_sequences=False),
    # Shape => [batch, out_steps*features].
    tf.keras.layers.Dense(OUT_STEPS*num_features,
                          kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features].
    tf.keras.layers.Reshape([OUT_STEPS, num_features])
])

print('Input shape:', multi_window.example[0].shape)
print('Output shape:', multi_lstm_model(multi_window.example[0]).shape)

multi_window.plot()
multi_window
multi_lstm_model.summary()

In [ ]:
MAX_EPOCHS = 20
history = compile_and_fit(multi_lstm_model, multi_window)
IPython.display.clear_output()
multi_lstm_model.summary()

In [ ]:
multi_val_performance = multi_lstm_model.evaluate(multi_window.val)
multi_performance = multi_lstm_model.evaluate(multi_window.test, verbose=0)
multi_window.plot(multi_lstm_model, max_subplots=5)

In [ ]:
plot_history(history.history)

In [ ]:
multi_performance

In [ ]:
#denormalised MAE
mae = (multi_performance[1] * df_denorm["load_kw_true"].std())
mae_pct = mae/df_denorm["load_kw_true"].mean()
mae_pct

In [ ]:
n_days = 7
y_actual = np.array(df[n-24*7:]).flatten()
y_pred = np.array([])
y_naive = np.array([])
for i in range(7, 0, -1):
    x = np.array(df.iloc[-i*24-7*24:-i*24]).reshape((1, 168, 1))
    y = tf.reshape(multi_lstm_model(x), (48,))[:24]
    y_pred = np.append(y_pred, y)
    y_naive = np.append(y_naive, np.array(df.iloc[-i*24-24:-i*24])) #y_naive = x of last 24 hours

plt.figure(figsize=(12,4))
x = np.array(df.index[n-24*7:])
plt.plot(x , y_actual)
plt.plot(x, y_pred)
plt.plot(x, y_naive)

plt.gca().xaxis.set_major_locator(mdates.HourLocator())
plt.gca().xaxis.set_major_locator(mdates.HourLocator(interval=180))
plt.gcf().autofmt_xdate()
plt.xticks(rotation=45)

plt.xlabel('Time')
plt.ylabel('Mean Absolute Error (MAE)')
plt.title('Performace of Long Short Term Memory (LSTM)')
plt.legend(['actual', 'prediction', 'naive'])

plt.show()

In [ ]:
# y_pred = x
mae_1 = sum(abs(y_actual-y_naive))/len(y_actual) * df_denorm["load_kw_true"].std()
mae_1_pct = mae_1/df_denorm["load_kw_true"].mean()
mae_1_pct

In [ ]:
# y_pres = mean_x
mae_2 = abs(np.array(df.iloc[-7*24-24:-7*24])-y_actual).mean()* df_denorm["load_kw_true"].std()
mae_2_pct = mae_2/df_denorm["load_kw_true"].mean()
mae_2_pct